## General Colab Tips
- Modify files by opening/editing them in the UI (double-click to open).
- `Right click > Refresh` in the Colab file explorer to update the directory.
- All files are lost when the Colab session disconnects, so make sure back up your work.
- Do **not** use `drive.mount` for your datasets! Reading from GDrive is super slow.
- Instead, place datasets into the `/content/` folder and modify your data accordingly.

**Make a copy of this notebook and modify this to whatever workflow you prefer!**

If you have some additional colab tips, please share them on the discussion forum.

## Setup

First, enable a GPU runtime via `Runtime > Change runtime type > T4 GPU`

Next, upload the your project files to the Colab. You can do this by either
- using Github (**recommended**)
- uploading files manually using the UI

## Github Setup

You can use git from within Google Colab!

For this section, we assume you know how to use git and have already pushed the starter code to a private repo.

Before you continue, make sure you download and push the starter code to your repo.  
It's a good idea to structure your repo something like
```
online_deep_learning/
    homework1/
    homework2/
    ...
```

We highly recommend using this workflow as you'll be able to easily pull/commit your changes after modifying your model on Colab.

To do this, you'll need a personal access token from [https://github.com/settings/tokens](https://github.com/settings/tokens)

The easiest thing to do is select "classic" token and make sure you have the `repo` scope selected to allow access to your private repos.
There's also fine-grained tokens where you can select access to specific repos.

Once you have your token, fill in your information and then run the following cell to clone your git repo to the Colab instance.

In [ ]:
import os


os.environ['USER'] = 'thisisanu'
os.environ['REPO'] = 'online_deep_learning'
os.environ['TOKEN'] = 'ithub_pat_11BIG54WA0sAoSEImPXImB_OUcAjo84vwWTugnGOH4GbPnqToNKRJESKhyxDDzjYYUDQXP7HGCBmOPHCVt'

# do everything in colab's "root" directory
%cd /content
!git clone https://${TOKEN}@github.com/${USER}/${REPO}.git

# make sure your repo shows up
%ls

## Code Setup

Move into `homework4/` so we can continue setting up the data / code for training.

This will be the main working directory and the training/grading must be run from this directory.


In [ ]:
# navigate to your repo
%cd /content/{os.environ['REPO']}
%ls

# go to a specific homework
%cd homework4
%ls


## Dataset Setup

Now that your code is all ready, the next step is to download the datasets.

Note: it's good practice to add data directories like `*/drive_data` to your `.gitignore` so you don't accidently commit them to your repo.

Since the datasets used in this class are relatively small, we can simply re-download them if the compute instance crashes/restarts.

In [ ]:
!rm -rf drive_data
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/drive_data.zip -o ./drive_data.zip && unzip -qo drive_data.zip
%ls

## Model Implementation + Training

Now you should be all set up.
Next, you'll need to implement
- `homework/train_planner.py`
- `homework/models.py`

And then you're ready to train

In [ ]:
# refreshes python imports automatically when you edit the source file
%load_ext autoreload
%autoreload 2

# make sure you're in the right directory
%pwd

from homework.train_planner import train


# your training args don't have to look like this
# this is just how it's structured in the solution
for lr in [1e-2, 1e-3, 1e-4]:
    train(
        model_name="linear_planner",
        transform_pipeline="state_only",
        num_workers=4,
        lr=lr,
        batch_size=128,
        num_epoch=40,
    )

## Grader

Run the following cell to grade your homework.

Note: if you don't set up PySuperTuxKart, the grader will not run the driving tests.

In [ ]:
!python3 -m grader homework -vv --disable_color

## PySuperTuxKart Setup (Optional)

We will use your trained planner to drive around in SuperTuxKart!  
SuperTuxKart is a python wrapper around a C++ game, so it requires a few more build steps.

This is optional to test locally - if your planner passes the local grader's tests for prediction accuracy, it should drive just fine and you can submit to the online grader. You only need to set up PySuperTuxKart locally if you want to see your model driving around.

In [ ]:
# don't worry about the "... is not a symbolic link" logs
!sudo DEBIAN_FRONTEND=noninteractive apt install -qq libnvidia-gl-535
!pip install PySuperTuxKartData --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk
!pip install PySuperTuxKart --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk

## End to End Driving Visualization

After your models are trained, you can see how they perform in game!

Some of the driving might look "jittery", due to the simple logic of the controller (directly steer towards the predicted waypoints).

You can always tweak the controller for fun, but note that we will use the original one for grading.

In [ ]:
# This script will simply run your model and save the session as a video
!python3 -m homework.supertux_utils.evaluate --model mlp_planner --track lighthouse --max-steps 100

In [ ]:
# This cell displays the videos
from IPython.display import Video
from pathlib import Path


video_dir = Path("videos")

for video_path in sorted(video_dir.glob("*.mp4")):
    print(video_path)
    display(Video(video_path, embed=True))

## Update your changes


In [ ]:
%ls
!git status

# Be careful not to "git add *" since there are datasets and logs
!git add homework/*.py
!git config --global user.email "GITHUB_EMAIL"
!git config --global user.name "GITHUB_USER"
!git commit -m "update"
!git push origin main

## Submission

Run the following cell to bundle your submission (modify UTID accordingly).

If you notice that your bundle is too large, you can modify the `bundle.py` script and ignore large files by adding them manually to `BLACKLIST`.

After the bundler and grader run, right click and download your bundled `.zip` file from the Colab UI.


In [ ]:
!python3 bundle.py homework UTID

# optional: run the grader with your bundled homework to double check
!python3 -m grader UTID.zip -vv --disable_color

## Tensorboard (Optional)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
!sudo DEBIAN_FRONTEND=noninteractive apt install -qq libnvidia-gl-535
!pip install PySuperTuxKartData --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk -vv
!pip install PySuperTuxKart --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk -vv

libnvidia-gl-535 is already the newest version (535.230.02-0ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-build-tracker-yc9ysclx
Initialized build tracking at /tmp/pip-build-tracker-yc9ysclx
Created build tracker: /tmp/pip-build-tracker-yc9ysclx
Entered build tracker: /tmp/pip-build-tracker-yc9ysclx
Created temporary directory: /tmp/pip-install-le6cxfk7
Created temporary directory: /tmp/pip-ephem-wheel-cache-1bs9gj98
Looking in indexes: https://www.cs.utexas.edu/~bzhou/dl_class/pystk
1 location(s) to search for versions of pysupertuxkartdata:
* https://www.cs.utexas.edu/~bzhou/dl_class/pystk/pysupertuxkartdata/
Fetching project page and analyzing links: https://www.cs.utexas.edu/~bzhou/dl_class/pystk/pysupertuxkartdata/
Getting page https://www.cs.utexas.edu/~bzhou/dl_class/pystk/pysupertuxkart

In [ ]:
!python -v

import _frozen_importlib # frozen
import _imp # builtin
import '_thread' # <class '_frozen_importlib.BuiltinImporter'>
import '_warnings' # <class '_frozen_importlib.BuiltinImporter'>
import '_weakref' # <class '_frozen_importlib.BuiltinImporter'>
import '_io' # <class '_frozen_importlib.BuiltinImporter'>
import 'marshal' # <class '_frozen_importlib.BuiltinImporter'>
import 'posix' # <class '_frozen_importlib.BuiltinImporter'>
import '_frozen_importlib_external' # <class '_frozen_importlib.FrozenImporter'>
# installing zipimport hook
import 'time' # <class '_frozen_importlib.BuiltinImporter'>
import 'zipimport' # <class '_frozen_importlib.FrozenImporter'>
# installed zipimport hook
# /usr/lib/python3.11/encodings/__pycache__/__init__.cpython-311.pyc matches /usr/lib/python3.11/encodings/__init__.py
# code object from '/usr/lib/python3.11/encodings/__pycache__/__init__.cpython-311.pyc'
import '_codecs' # <class '_frozen_importlib.BuiltinImporter'>
import 'codecs' # <class '_frozen_impor